## CLEANUP AND DISTANCE MAPPING
* Code will get rid of any listing records with 0 reviews
* code will calculate distance of listing from Melbourne CBD
* code will calculate distance of listing from train station

In [3]:
# import libraries
import math
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_columns', None)

In [4]:


def get_distance(latA,longA,latB,longB)  :
    # radius of the Earth
    R = 6373.0

    #coordinates
    lat1 = math.radians(latA)
    lon1 = math.radians(longA)
    lat2 = math.radians(latB)
    lon2 = math.radians(longB)

    # change in coordinates
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    #  Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2


    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c

    return(distance)

In [7]:
# import listings 2018 csv to pd dataframe
path="../2018/listings_2018_prelimClean_JS.csv"
airbnb_df = pd.read_csv(path, low_memory=False)

22895

In [8]:
airbnb_df.head()


,id,scrape_id,last_scraped,name,summary,neighborhood_overview,transit,access,interaction,host_id,host_name,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,9835,2.020000e+13,7/12/18,Beautiful Room & House,NaN,Very safe! Family oriented. Older age group.,"YES ! The bus (305,309) is exactly two blocks ...","Kitchen, backyard, upstairs lounge. We'd like ...",As much as the guest wants.,33057,Manju,21/8/09,AU,a few days or more,0%,NaN,f,NaN,1.0,1.0,"Bulleen, VIC, Australia",Balwyn North,Manningham,NaN,Bulleen,VIC,3105,Melbourne,"Bulleen, Australia",AU,Australia,-37.772684,145.092133,f,House,Private room,2,1.0,1.0,2.0,Real Bed,"{Internet,Wifi,""Pets live on this property"",Do...",NaN,$60.00,NaN,NaN,NaN,NaN,1,$22.00,1,365,17 months ago,t,30,60,90,365,4,24/5/11,12/9/15,90.0,9.0,10.0,10.0,10.0,9.0,9.0,0.04
1,10803,2.020000e+13,7/12/18,Room in Cool Deco Apartment in Brunswick,A large air conditioned room with queen spring...,This hip area is a crossroads between two grea...,Easy transport options - the tram is right out...,Wifi. Bathroom and kitchen is shared but I mos...,I'm home mostly but sometimes I may be working...,38901,Lindsay,16/9/09,"Melbourne, Victoria, Australia",NaN,NaN,NaN,f,Brunswick,1.0,1.0,"Brunswick East, VIC, Australia",Brunswick,Moreland,NaN,Brunswick East,VIC,3057,Melbourne,"Brunswick East, Australia",AU,Australia,-37.766505,144.980736,t,Apartment,Private room,2,1.0,1.0,1.0,Futon,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,H...",226.0,$35.00,$200.00,$803.00,$0.00,$9.00,1,$15.00,3,45,6 weeks ago,t,4,9,33,213,108,12/1/13,26/11/18,87.0,9.0,8.0,9.0,9.0,9.0,9.0,1.50
2,12936,2.020000e+13,7/12/18,St Kilda 1BR APT+BEACHSIDE+VIEWS+PARKING+WIFI+AC,RIGHT IN THE HEART OF ST KILDA! It doesn't get...,A stay at our apartment means you can enjoy so...,Our apartment is located within walking distan...,Guests have exclusive and private access to th...,We don't always get to meet all our guests in ...,50121,Frank & Vince,31/10/09,"Melbourne, Victoria, Australia",within an hour,100%,NaN,t,St Kilda,17.0,17.0,"St Kilda, VIC, Australia",St Kilda,Port Phillip,NaN,St Kilda,VIC,3182,Melbourne,"St Kilda, Australia",AU,Australia,-37.859755,144.977369,t,Apartment,Entire home/apt,3,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",0.0,$159.00,"$1,253.00","$4,452.00",$450.00,$100.00,2,$28.00,3,28,today,t,10,26,51,326,15,4/8/10,7/11/18,94.0,9.0,10.0,9.0,9.0,9.0,9.0,0.15
3,15246,2.020000e+13,7/12/18,Large private room-close to city,"Comfortable, relaxed house, a home away from ...","This is a great neighbourhood â€“ it is quiet,...",Public transport is super convenient with a ch...,You are welcome to make yourself at home in th...,Guest's privacy will be respected at all times...,59786,Eleni,3/12/09,"Melbourne, Victoria, Australia",NaN,NaN,NaN,f,Thornbury,3.0,3.0,"Thornbury, VIC, Australia",Thornbury,Darebin,NaN,Thornbury,VIC,3071,Melbourne,"Thornbury, Australia",AU,Australia,-37.758971,144.989228,t,House,Private room,1,1.5,1.0,1.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Free parking on pre...",0.0,$50.00,$250.00,$920.00,$200.00,$15.00,1,$20.00,2,200,11 months ago,t,0,0,0,0,29,31/12/10,15/5/17,94.0,9.0,9.0,

In [19]:
airbnb_df2 = (airbnb_df.loc[airbnb_df['room_type'] == 'Entire home/apt']
             .loc[airbnb_df['number_of_reviews'] > 0])
# feel free to comment  fields we do not need or uncomment  fields we do
airbnb_df2 = airbnb_df2[
            ['id',
            #'scrape_id', #i dont see a significance of scrape ID
            #'last_scraped', #shows only 2 dates, dec 7 and 8 most likely when the data was last updated
            'name',
            #'summary',
            'neighborhood_overview',
            'transit',
            #'access',
            #'interaction',
            'host_id',
            'host_name',
            'host_since',
            'host_location',
            #'host_response_time',
            #'host_response_rate',
            #'host_acceptance_rate',
            'host_is_superhost',
            'host_neighbourhood', 
            'host_listings_count',
            'host_total_listings_count',
            'street',
            #'neighbourhood', # neighbourhood_cleansed shows a more accurate location
            'neighbourhood_cleansed',
            #'neighbourhood_group_cleansed',
            'city',
            'state',
            'zipcode',
            #'market', #we know everything should be in melbourne plus there are confusing markets
            #'smart_location', #ucomment if needed
            #'country_code', #we know everything is in AU
            #'country', #we know everything is in AU
            'latitude',
            'longitude',
            'is_location_exact',
            'property_type',
            #'room_type', # no need fo this field since we are only using 'Entire home/apt'
            'accommodates',
            'bathrooms',
            'bedrooms',
            'beds',
            'bed_type',
            'amenities',
            'square_feet',
            'price',
            'weekly_price',
            'monthly_price',
            'security_deposit',
            'cleaning_fee',
            'guests_included',
            'extra_people',
            'minimum_nights',
            'maximum_nights',
            'calendar_updated',
            'has_availability',
            'availability_30',
            'availability_60',
            'availability_90',
            'availability_365',
            'number_of_reviews',
            'first_review',
            'last_review',
            'review_scores_rating',
            'review_scores_accuracy',
            'review_scores_cleanliness',
            'review_scores_checkin',
            'review_scores_communication',
            'review_scores_location',
            'review_scores_value',
            'reviews_per_month']]

airbnb_df2['distance_from_cbd'] = ""
airbnb_df2

,id,name,neighborhood_overview,transit,host_id,host_name,host_since,host_location,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,street,neighbourhood_cleansed,city,state,zipcode,latitude,longitude,is_location_exact,property_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,distance_from_cbd
2,12936,St Kilda 1BR APT+BEACHSIDE+VIEWS+PARKING+WIFI+AC,A stay at our apartment means you can enjoy so...,Our apartment is located within walking distan...,50121,Frank & Vince,31/10/09,"Melbourne, Victoria, Australia",t,St Kilda,17.0,17.0,"St Kilda, VIC, Australia",Port Phillip,St Kilda,VIC,3182,-37.859755,144.977369,t,Apartment,3,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",0.0,$159.00,"$1,253.00","$4,452.00",$450.00,$100.00,2,$28.00,3,28,today,t,10,26,51,326,15,4/8/10,7/11/18,94.0,9.0,10.0,9.0,9.0,9.0,9.0,0.15,
5,38271,Melbourne - Old Trafford Apartment,Our street is quiet & secluded but within walk...,If you're arriving via the airport (Tullamarin...,164193,Daryl,12/7/10,"Berwick, Victoria, Australia",t,NaN,1.0,1.0,"Berwick, VIC, Australia",Casey,Berwick,VIC,3806,-38.057254,145.339356,t,Apartment,5,1.0,3.0,3.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Pool,Kitc...",NaN,$98.00,$540.00,NaN,$0.00,$0.00,1,$30.00,1,14,2 weeks ago,t,12,27,57,328,112,24/11/10,1/12/18,97.0,10.0,10.0,10.0,10.0,10.0,10.0,1.14,
8,43429,Tranquil Javanese-Style Apartment in Oakleigh ...,Oakleigh is one of the most convenient and div...,"Monash University, Clayton Campus is a 10 minu...",189684,Allan,3/8/10,"Melbourne, Victoria, Australia",t,NaN,2.0,2.0,"Oakleigh East, VIC, Australia",Monash,Oakleigh East,VIC,3166,-37.899601,145.114475,t,Apartment,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",388.0,$98.00,$535.00,NaN,$500.00,$40.00,1,$0.00,2,365,today,t,7,25,35,310,168,5/12/10,3/12/18,97.0,10.0,10.0,10.0,10.0,10.0,10.0,1.72,
10,47100,TrÃ¨s Charming in Fabulous Richmond,Bridge Road (once famous for discount shopping...,There are two tram lines right outside the apa...,212071,Loren,24/8/10,"Melbourne, Victoria, Australia",t,Richmond,4.0,4.0,"Richmond, VIC, Australia",Yarra,Richmond,VIC,3121,-37.818037,145.004416,t,Apartment,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",NaN,$98.00,$800.00,NaN,$200.00,$80.00,1,$20.00,4,30,2 weeks ago,t,13,13,18,22,176,9/9/10,1/11/18,98.0,10.0,10.0,10.0,10.0,10.0,10.0,1.75,
11,50492,St Kilda Central LUXE 2BR APT+TWO COURTYARDS+WIFI,St Kilda is one of Melbourneâ€™s most vibrant ...,Our apartment is located within walking distan...,50121,Frank & Vince,31/10/09,"Melbourne, Victoria, Australia",t,St Kilda,17.0,17.0,"St Kilda, VIC, Australia",Port Phillip,St Kilda,VIC,3182,-37.861275,144.980379,t,Apartment,4,1.0,2.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Buzzer/wireless int...",0.0,$190.00,"$1,743.00","$5,572.00",$500.00,$110.00,1,$0.00,3,28,today,t,10,22,44,316,44,30/11/12,2/12/18,97.0,10.0,10.0,10.0,10.0,9.0,10.0,0.60,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22545,30412315,Doncaster Modern Oasis. Netflix / Pool / Break...,Doncaster is perfectly placed to explore many ...,With a large bus station at Westfield Shopping...,24980853,Paul,18/12/14,"Melbourne, Victoria, Australia",f,NaN,2.0,2.0,"Doncaster, VIC, Australia",Manningham,Doncaster,VIC,3108,-37.782288,145.127

In [21]:
# add distance from city in KMs
cbdLat =-37.814326
cbdLong=144.963149

for index, row in airbnb_df2.iterrows():
    distance = get_distance(row['latitude'], #latA
                            row['longitude'], #longA
                        cbdLat, #latB
                        cbdLong) #longB
    airbnb_df2.loc[index, 'distance_from_cbd'] = distance
airbnb_df2

,id,name,neighborhood_overview,transit,host_id,host_name,host_since,host_location,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,street,neighbourhood_cleansed,city,state,zipcode,latitude,longitude,is_location_exact,property_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,distance_from_cbd
2,12936,St Kilda 1BR APT+BEACHSIDE+VIEWS+PARKING+WIFI+AC,A stay at our apartment means you can enjoy so...,Our apartment is located within walking distan...,50121,Frank & Vince,31/10/09,"Melbourne, Victoria, Australia",t,St Kilda,17.0,17.0,"St Kilda, VIC, Australia",Port Phillip,St Kilda,VIC,3182,-37.859755,144.977369,t,Apartment,3,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",0.0,$159.00,"$1,253.00","$4,452.00",$450.00,$100.00,2,$28.00,3,28,today,t,10,26,51,326,15,4/8/10,7/11/18,94.0,9.0,10.0,9.0,9.0,9.0,9.0,0.15,5.20517
5,38271,Melbourne - Old Trafford Apartment,Our street is quiet & secluded but within walk...,If you're arriving via the airport (Tullamarin...,164193,Daryl,12/7/10,"Berwick, Victoria, Australia",t,NaN,1.0,1.0,"Berwick, VIC, Australia",Casey,Berwick,VIC,3806,-38.057254,145.339356,t,Apartment,5,1.0,3.0,3.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Pool,Kitc...",NaN,$98.00,$540.00,NaN,$0.00,$0.00,1,$30.00,1,14,2 weeks ago,t,12,27,57,328,112,24/11/10,1/12/18,97.0,10.0,10.0,10.0,10.0,10.0,10.0,1.14,42.6539
8,43429,Tranquil Javanese-Style Apartment in Oakleigh ...,Oakleigh is one of the most convenient and div...,"Monash University, Clayton Campus is a 10 minu...",189684,Allan,3/8/10,"Melbourne, Victoria, Australia",t,NaN,2.0,2.0,"Oakleigh East, VIC, Australia",Monash,Oakleigh East,VIC,3166,-37.899601,145.114475,t,Apartment,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",388.0,$98.00,$535.00,NaN,$500.00,$40.00,1,$0.00,2,365,today,t,7,25,35,310,168,5/12/10,3/12/18,97.0,10.0,10.0,10.0,10.0,10.0,10.0,1.72,16.3273
10,47100,TrÃ¨s Charming in Fabulous Richmond,Bridge Road (once famous for discount shopping...,There are two tram lines right outside the apa...,212071,Loren,24/8/10,"Melbourne, Victoria, Australia",t,Richmond,4.0,4.0,"Richmond, VIC, Australia",Yarra,Richmond,VIC,3121,-37.818037,145.004416,t,Apartment,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",NaN,$98.00,$800.00,NaN,$200.00,$80.00,1,$20.00,4,30,2 weeks ago,t,13,13,18,22,176,9/9/10,1/11/18,98.0,10.0,10.0,10.0,10.0,10.0,10.0,1.75,3.64957
11,50492,St Kilda Central LUXE 2BR APT+TWO COURTYARDS+WIFI,St Kilda is one of Melbourneâ€™s most vibrant ...,Our apartment is located within walking distan...,50121,Frank & Vince,31/10/09,"Melbourne, Victoria, Australia",t,St Kilda,17.0,17.0,"St Kilda, VIC, Australia",Port Phillip,St Kilda,VIC,3182,-37.861275,144.980379,t,Apartment,4,1.0,2.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Buzzer/wireless int...",0.0,$190.00,"$1,743.00","$5,572.00",$500.00,$110.00,1,$0.00,3,28,today,t,10,22,44,316,44,30/11/12,2/12/18,97.0,10.0,10.0,10.0,10.0,9.0,10.0,0.60,5.437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22545,30412315,Doncaster Modern Oasis. Netflix / Pool / Break...,Doncaster is perfectly placed to explore many ...,With a large bus station at Westfield Shopping...,24980853,Paul,18/12/14,"Melbourne, Victoria, Australia",f,NaN,2.0,2.0,"Doncaster, VIC, Australia",Manningham,Donc

In [30]:
max_distance = 12

airbnb_df3 = airbnb_df2.loc[airbnb_df2['distance_from_cbd'] <= max_distance]

# save data to CSV
airbnb_df3.to_csv('listings_2018_clean2_RS.csv')
airbnb_df3

,id,name,neighborhood_overview,transit,host_id,host_name,host_since,host_location,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,street,neighbourhood_cleansed,city,state,zipcode,latitude,longitude,is_location_exact,property_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,distance_from_cbd
2,12936,St Kilda 1BR APT+BEACHSIDE+VIEWS+PARKING+WIFI+AC,A stay at our apartment means you can enjoy so...,Our apartment is located within walking distan...,50121,Frank & Vince,31/10/09,"Melbourne, Victoria, Australia",t,St Kilda,17.0,17.0,"St Kilda, VIC, Australia",Port Phillip,St Kilda,VIC,3182,-37.859755,144.977369,t,Apartment,3,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",0.0,$159.00,"$1,253.00","$4,452.00",$450.00,$100.00,2,$28.00,3,28,today,t,10,26,51,326,15,4/8/10,7/11/18,94.0,9.0,10.0,9.0,9.0,9.0,9.0,0.15,5.20517
10,47100,TrÃ¨s Charming in Fabulous Richmond,Bridge Road (once famous for discount shopping...,There are two tram lines right outside the apa...,212071,Loren,24/8/10,"Melbourne, Victoria, Australia",t,Richmond,4.0,4.0,"Richmond, VIC, Australia",Yarra,Richmond,VIC,3121,-37.818037,145.004416,t,Apartment,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",NaN,$98.00,$800.00,NaN,$200.00,$80.00,1,$20.00,4,30,2 weeks ago,t,13,13,18,22,176,9/9/10,1/11/18,98.0,10.0,10.0,10.0,10.0,10.0,10.0,1.75,3.64957
11,50492,St Kilda Central LUXE 2BR APT+TWO COURTYARDS+WIFI,St Kilda is one of Melbourneâ€™s most vibrant ...,Our apartment is located within walking distan...,50121,Frank & Vince,31/10/09,"Melbourne, Victoria, Australia",t,St Kilda,17.0,17.0,"St Kilda, VIC, Australia",Port Phillip,St Kilda,VIC,3182,-37.861275,144.980379,t,Apartment,4,1.0,2.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Buzzer/wireless int...",0.0,$190.00,"$1,743.00","$5,572.00",$500.00,$110.00,1,$0.00,3,28,today,t,10,22,44,316,44,30/11/12,2/12/18,97.0,10.0,10.0,10.0,10.0,9.0,10.0,0.60,5.437
12,51592,Central City Warehouse Apartment,The apartment is located in the centre of Melb...,"Yes, minutes to trains, trams and buses but mo...",190879,Michelle,4/8/10,"Victoria, Australia",t,Central Business District,1.0,1.0,"Melbourne, VIC, Australia",Melbourne,Melbourne,VIC,3000,-37.814856,144.962672,t,Loft,4,1.5,1.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Paid parking off pr...",NaN,$228.00,"$1,575.00","$5,100.00",$250.00,$40.00,2,$40.00,2,90,3 weeks ago,t,13,39,56,331,196,22/2/11,1/12/18,99.0,10.0,9.0,10.0,10.0,10.0,10.0,2.07,0.072335
16,66754,Richmond Super Central 60s Cool 1BR APT+WIFI+AC,A stay at our hip apartment will mean you can ...,Our apartment is located within walking distan...,50121,Frank & Vince,31/10/09,"Melbourne, Victoria, Australia",t,St Kilda,17.0,17.0,"Richmond, VIC, Australia",Yarra,Richmond,VIC,3121,-37.820916,144.993839,t,Apartment,4,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",0.0,$138.00,"$1,183.00","$4,172.00",$450.00,$100.00,2,$28.00,3,28,today,t,11,19,49,324,43,8/3/11,11/11/18,95.0,9.0,10.0,10.0,10.0,10.0,9.0,0.46,2.79455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22537,30410904,Melbourne CBD 2 Bedrooms/å¢¨å°”æœ¬å¸‚ä¸­å¿ƒ2æˆ...,NaN,NaN,127327853,Nathan,24/4/17,"Melbourne, Victoria, Australia",f,Central Business District,1.0,1.0,"Melbourne, VIC, Australia",Melbourne,Melbourne,VIC,3000,-37.80871

In [28]:
#load station data
path2="./ptv_metro_train_station.csv"
stations_df = pd.read_csv(path2)
stations_df

,STOP_ID,STOP_NAME,LATITUDE,LONGITUDE,TICKETZONE,ROUTEUSSP
0,19967,Anstey Railway Station (Brunswick),-37.761242,144.960684,1,Upfield
1,19968,Brunswick Railway Station (Brunswick),-37.767721,144.959587,1,Upfield
2,19969,Jewell Railway Station (Brunswick),-37.774987,144.958717,1,Upfield
3,19970,Royal Park Railway Station (Parkville),-37.781193,144.952301,1,Upfield
4,19971,Flemington Bridge Railway Station (North Melbo...,-37.788140,144.939323,1,Upfield
...,...,...,...,...,...,...
216,19962,Fawkner Railway Station (Fawkner),-37.714621,144.960545,"1,2",Upfield
217,19963,Merlynston Railway Station (Coburg North),-37.720934,144.961314,"1,2",Upfield
218,19964,Batman Railway Station (Coburg North),-37.733523,144.962841,"1,2",Upfield
219,19965,Coburg Railway Station (Coburg),-37.742345,144.963336,1,Upfield


In [ ]:
# nearest station
#airbnb_df3['nearest_station']=""
#airbnb_df3['station_distance']=""

#for index, listing in airbnb_df3.iterrows():
#    stop_name=[]
#    distance=[]
#    stations_df2 = stations_df.copy()
#    for index, station in stations_df2.iterrows():
#        
#        distance = get_distance(listing['latitude'], #latA
#                                listing['longitude'], #longA
#                        cbdLat, #latB
#                        cbdLong) #longB
        